In [10]:
#for plan and executor
!pip install langchain_experimental
# for serpAPI
!pip install google-search-results 

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32003 sha256=b5a09359a9606a54c1f692b8a53ba082adbaa47933088bac2bef043b351ce94e
  Stored in directory: /Users/terrycho/Library/Caches/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [7]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain

# define tools

In [14]:
# SearpAPI needs API key, you can sign in the site and api key https://serpapi.com/users/welcome
search = SerpAPIWrapper(serpapi_api_key="373a2e2893aa390091fcc7c15e90f4f8434f888d89794d30a4f40dedf8b57648")
llm = OpenAI(temperature=0,openai_api_key="{YOUR_OPENAI_KEY}")
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

model = ChatOpenAI(openai_api_key="{YOUR_OPENAI_KEY}",temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new PlanAndExecute chain...
steps=[Step(value="Search for information about Leo DiCaprio's girlfriend."), Step(value='Find her current age.'), Step(value='Calculate her current age raised to the 0.43 power.'), Step(value="Given the above steps taken, respond to the user's original question.\n\n")]

> Entering new AgentExecutor chain...
Thought: I can use the Search tool to find information about Leo DiCaprio's girlfriend.

Action:
```{"action": "Search", "action_input": "Leo DiCaprio girlfriend"}```

> Finished chain.
*****

Step: Search for information about Leo DiCaprio's girlfriend.

Response: Thought: I can use the Search tool to find information about Leo DiCaprio's girlfriend.

Action:
```{"action": "Search", "action_input": "Leo DiCaprio girlfriend"}```

> Entering new AgentExecutor chain...
Thought: To find Leo DiCaprio's girlfriend's current age, I can search for information about her and see if her age is mentioned. Let me use the Search tool to find this informa

"Leo DiCaprio's girlfriend is Camila Morrone."